# 과제 목표

1. 텍스트 데이터를 토큰화하는 방법 학습
2. GPT 모델의 특성 및 구조를 이해하고 구현

> 본 실습은 Andrej Karpathy의 [Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=0s) 영상 속 실습 코드 일부를 포함하고 있습니다.

# 환경 세팅

본 실습에서 사용할 라이브러리를 import 하겠습니다.

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

본 실습에 사용할 설정 값을 지정하겠습니다. 데이터셋 내 vocabulary 크기가 필요하기 때문에 Config의 할당은 추후에 수행합니다.

In [2]:
class Config:
    def __init__(self, vocab_size):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.max_iters = 10000
        self.learning_rate = 1e-4
        self.batch_size = 64
        self.n_dec_vocab = vocab_size # size of vocabulary
        self.seq_len = 32 # maximum length of decoder prediction sequence
        self.n_layer = 4 # # number of decoder layers
        self.n_head = 4 # number of heads
        self.d_head = 64 # dimension of heads
        self.d_model = 256 # dimension of model. n_head * d_head
        self.d_ff = 1024 # dimension of feed forward layer
        self.dropout = 0.0 # dropout rate
        self.layer_norm = 1e-3 # layer normalization epsilon

본 실습에서는 셰익스피어의 글로 이루어진 짧은 데이터셋을 불러와 사용하겠습니다. 아래 명령어로 데이터셋을 다운 받습니다.

In [5]:
# 아래 두 명령어 중 작동하는 것으로 사용
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt

--2024-09-04 23:59:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-09-04 23:59:00 (33.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



파일을 열어 텍스트를 불러옵니다.

In [6]:
# Load dataset
with open('input.txt', 'r', encoding='utf-8') as f:
    dataset = f.read()

# GPT Pre-training 구현

## 1. 텍스트 토큰화 및 인코딩/디코딩

보통의 LLM에서는 sub-word 단위의 토큰을 사용하나, 구현의 편리함을 위하여 본 실습에서는 문자(character) 단위로 토큰화를 하겠습니다.

먼저, 데이터셋 내 문자의 종류와 개수를 파악합니다. 또한 문자와 토큰 인덱스(정수)을 서로 전환할 수 있는 인코더, 디코더를 만들고, 데이터셋을 토큰화 및 인코딩 하겠습니다.

In [7]:
############################################################################
# Req 3-1: 텍스트 토큰화 및 인코딩/디코딩                                      #
############################################################################

################################################################################
# TODO: 데이터셋 내 문자의 종류 및 개수를 파악하고, 문자 <-> 토큰 인덱스(정수)를 전환할   #
# 인코더와 디코더를 만들고, 데이터셋을 토큰화 & 인코딩함                               #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) 데이터셋 내 문자의 종류를 리스트로 만듦
# 2) 문자를 토큰 인덱스(정수)로, 토큰 인덱스(정수)를 문자로 매핑할 수 있는 딕셔너리를 만듦.
# 3) 위 딕셔너리를 사용하여 인코더와 디코더를 만듦.
#    인코더는 문장 입력을 토큰 인덱스의 리스트로 변환하고, 디코더는 그 반대 역할을 함.

# 데이터셋 내 문자의 종류를 리스트로 만듦
characters = sorted(set(dataset))

# Size of vocabulary (= number of characters)
vocab_size = len(characters)

# 문자를 토큰 인덱스(정수)로, 토큰 인덱스(정수)를 문자로 매핑할 수 있는 딕셔너리를 만듦.
# 전자는 key가 문자, value가 토큰 인덱스(정수)이고, 후자는 key가 토큰 인덱스(정수), value가 문자임
str_to_int = {char: idx for idx, char in enumerate(characters)}
int_to_str = {idx: char for idx, char in enumerate(characters)}

# 위 딕셔너리를 사용하여 인코더와 디코더를 만듦.
# 인코더는 문장 입력을 토큰 인덱스의 리스트로 변환하고, 디코더는 그 반대 역할을 함.
def encode(str_to_int, sentence):
    """Return list of token indices"""
    return [str_to_int[char] for char in sentence]

def decode(int_to_str, indices):
    """Return the original string"""
    return ''.join([int_to_str[idx] for idx in indices])

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

아래 코드로 위 구현 내용을 테스트할 수 있습니다.

In [8]:
print(f"All tokens (including space and new line): {''.join(characters)}")
print(f"Size of Vocab: {vocab_size}")

ex_str = "I am a boy."
enc_result = encode(str_to_int, ex_str)
dec_result = decode(int_to_str, enc_result)
print(f"Encoding example: \'{ex_str}\' -> {enc_result}")
print(f"Decoding example: \'{enc_result}\' -> {dec_result}")

All tokens (including space and new line): 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Size of Vocab: 65
Encoding example: 'I am a boy.' -> [21, 1, 39, 51, 1, 39, 1, 40, 53, 63, 8]
Decoding example: '[21, 1, 39, 51, 1, 39, 1, 40, 53, 63, 8]' -> I am a boy.


데이터셋을 인코딩하면 아래와 같습니다.

In [9]:
train_data = torch.tensor(encode(str_to_int, dataset), dtype=torch.long)

In [10]:
print(f"Size of training data: {train_data.shape}")
print("Example of training data")
print(train_data[:100])

Size of training data: torch.Size([1115394])
Example of training data
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Vocabulary의 크기를 알게 되었으므로 설정값을 불러옵니다.

In [11]:
cfg = Config(vocab_size)

## 2. Batch 단위로 데이터셋 나누기

현재 데이터셋은 토큰의 나열로 되어 있습니다. 모델에서 학습, 추론에 사용하려면 이들을 시퀀스, 배치 단위로 나눠야 합니다. 해당 실습에서는 이전 단어를 주었을 때 다음 단어를 예측하는 모델을 학습할 것입니다. 정답이 되는 단어는 다음 단어입니다.

- Full sequence: [12, 34, 15, 18, 92] 라고 하면
- input : [12, 34, 15] / target: 18
- input : [12, 34, 15, 18] / target: 92

시퀀스 및 배치 단위로 나누면 예시는 아래와 같습니다. (배치 크기: 4, 시퀀스 최대 길이: 8)

```py
inputs: shape - [4, 8]
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: shape - [4, 8]
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
```

앞선 설명을 참고하여, 데이터셋이 주어졌을 때 이를 배치 단위의 모델 입력을 만드는 함수를 구현하겠습니다.

In [ ]:
############################################################################
# Req 3-2: Batch 단위로 데이터셋 나누기                                        #
############################################################################

def get_batch(dataset, seq_len, batch_size):
    """
    Args:
        dataset (torch.Tensor): Tokenized text dataset
        seq_len (int): maximum length of each sequence (i.e., sentence)
        batch_size (int): number of sequences in each batch
    Returns:
        data (torch.Tensor): batched input data
        label (torch.Tensor): batched target (label) data
    """
    ################################################################################
    # TODO: 배치를 구성할 시작 위치를 특정하고, 모델의 입력 데이터와 그에 대응하는           #
    # 라벨 데이터(한 문자씩 뒤로 미룸)를 뽑음                                           #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # 1) 데이터 split 내 어디서부터 seq_len만큼 입력 시퀀스를 뽑을지 시작 인덱스를 랜덤하게 뽑음.
    #    배치 크기만큼의 시작 인덱스를 뽑아 리스트로 구성함.
    # 2) 모델의 입력 데이터로, (1)의 각 시작 인덱스에 대해 입력 데이터를 seq_len 만큼 뽑아 저장함.
    # 3) 대응하는 라벨(타겟)데이터로, (2)와 동일한 과정으로 정답 데이터를 저장하되, (2)보다 한 인덱스씩 차이가 나도록 추출함.

    """Write your code"""

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################

    return data, label

아래 코드로 `get_batch` 함수를 테스트할 수 있습니다.

In [ ]:
data, label = get_batch(dataset=train_data[:100], seq_len=16, batch_size=2)
print("data")
print(data)
print("label")
print(label)

## 3. Masked self-attention 구현하기

본 실습에서 구현할 GPT는 크게는 아래와 같은 구조로 되어 있습니다.

```py
GPT
  └ DecoderLayer (x n_layer 개)
     ├ MultiHeadAttention
     │  └ ScaledDotProdAttn
     └ PositionwiseFeedForward
```

디코더 레이어를 쌓아 GPT를 만들며, 각 디코더 레이어는 Multi-head self-attention과 Feedforwad Network 등으로 구성됩니다. Multi-head attention은 정답 부분을 가려 모델의 학습을 돕는 masked self-attention으로 구현하며, 본 실습 구현에선 Scaled dot-product에서 마스킹을 진행합니다.

GPT는 단방향 언어 모델로, 이전 시퀀스(1 ~ t-1)만 보고 다음(t)에 올 단어를 예측합니다. 따라서 t부터는 정답에 해당합니다.

데이터 배치 내엔 타겟하는 단어와 그 이후 단어까지 모두 나타나 있습니다. 그대로 self-attention을 수행한다면 모델은 정답도 보고 학습하게 되어 제대로 된 학습이 되지 않습니다. 따라서 입력, 즉 이전 시퀀스만 보도록 마스킹을 하는, Masked self-attention을 구현하겠습니다.

`tensor.masked_fill` 메서드는 지정한 인덱스에 지정한 값으로 마스킹을 합니다. 우리는 한 배치 내 첫 시퀀스는 한 개, 두 번째 시퀀스는 두 개, ... 식으로 입력으로 삼고 그 뒤는 "-inf" 값으로 마스킹을 할 것입니다. 아래는 예시입니다.

```py
data = [
    [98, 45, 87, 22, 89],
    [45, 11, 80, 14, 45],
    [66, 73, 93, 74, 12],
    [10, 26, 63, 33, 41],
    [98, 68, 90, 64, 95],
]
mask = [
    [True, False, False, False, False],
    [True, True,  False, False, False],
    [True, True,  True,  False, False],
    [True, True,  True,  True,  False],
    [True, True,  True,  True,  True],
]

>> data.masked_fill(mask==0, float('-inf'))
[
    [98, -inf, -inf, -inf, -inf],
    [45,   11, -inf, -inf, -inf],
    [66,   73,   93, -inf, -inf],
    [10,   26,   63,   33, -inf],
    [98,   68,   90,   64,   95],
]
```

In [ ]:
############################################################################
# Req 3-3: Masked self-attention 구현하기                                    #
############################################################################

class ScaledDotProdAttn(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.dropout = nn.Dropout(config.dropout)
        self.scale = 1 / math.sqrt(self.config.d_head)

        # Masking을 위한 삼각행렬. 입력 데이터의 각 배치와 동일한 크기를 갖게 선언.
        self.tril = torch.tril(
            torch.ones(
                self.config.n_head,
                self.config.seq_len,
                self.config.seq_len,
            )
        ).to(self.config.device)

    def forward(self, q, k, v):
        """
        Scaled dot product attention with masking

        Args:
            q, k, v (torch.Tensor): Query, Key, Value Matrices. shape - (B, n_head, seq_len, d_head)

        Returns:
            attn_values (torch.Tensor): Attention results. shape - (B, n_head, seq_len, d_head)
        """
        ################################################################################
        # TODO: Sclaed dot-product attention 연산을 하되, Query와 Key를 이용하여 구한       #
        # attention score에 masking하여 연산함.                                          #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근(self.scale)으로 나눔 (=(Q x K^T) / sqrt(d_head))
        # 2) masked_fill 메서드와 self.tril 값을 이용하여 (1)에서 구한 값에 마스킹을 진행함. tril이 0인 곳에 -inf로 마스킹함.
        # 3) (2)의 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
        # 4) (3)에 dropout을 적용함
        # 5) Value 벡터를 곱해 최종 attention value 계산

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return attn_values

## 4. Multi-head Self-Attention 클래스 구현하기

Sub PJT 1의 실습처럼 multi-head self-attention을 구현하겠습니다. 과정은 아래와 같이 요약할 수 있습니다.

1. 입력 데이터를 q, k, v 로 linear projection
2. Scaled dot-product attention
    - 앞선 `ScaledDotProdAttn` 클래스 이용
3. Concatination of heads' output
4. Linear projection & Dropout

In [ ]:
############################################################################
# Req 3-4: Multi-head Self-Attention 클래스 구현하기                          #
############################################################################

class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config

        ################################################################################
        # TODO: 조건에 맞게 모델의 레이어를 정의함                                          #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 임베딩 벡터를 Q, K, V 벡터로 변환할 레이어 정의
        # 2) Scaled Dot-Product Attention 모듈 선언
        # 3) 각 헤드의 결과에 가중치를 부여할 linear layer 정의
        # 4) Dropout 레이어 정의

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################


    def forward(self, x):
        """
        Multi-head attention with masking

        Args:
            x (torch.Tensor): shape - (B, seq_len, d_model(=n_head*d_head))
        Returns:
            output (torch.Tensor): shape - (B, seq_len, d_model(=n_head*d_head))
        """
        ################################################################################
        # TODO: Multi-head self-attention 과정을 정의함                                  #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 입력 데이터를 Q, K, V 벡터로 변환하고, Head의 수로 분할한 뒤, 각 head가 (L, d_k)의 matrix를 담당하도록 만듦
        # 2) Scaled dot-product self-attention 연산을 수행함
        # 3) 각 attention head의 결과물을 concatenate해 병합함
        # 4) head 별로 정해진 가중치로 linear projection함
        # 5) Dropout을 적용함

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return output

## 5. FeedForward Network 정의하기

어텐션 후 적용되는 FFN은 비선형성을 추가하고, 고차원으로 latent space를 확장함으로써 더 복잡한 특징을 학습할 수 있도록 돕습니다.

FFN은 아래와 같이 구성되어 있고 이 순서로 작동합니다.

- 첫 번째 선형 레이어: `d_model` 차원 -> `d_ff` 차원
- 활성화 함수 (ReLU)
- 두 번째 선형 레이어: `d_ff` 차원 -> `d_model` 차원
- Dropout

In [ ]:
############################################################################
# Req 3-5: FeedForward Network 정의하기                                      #
############################################################################

################################################################################
# TODO: __init__()에 조건에 맞게 모델의 레이어를 정의하고,                           #
# forward()에 FFN 연산을 정의함                                                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) __init__()에 선형 레이어 2개, ReLU 활성화 함수 1개, dropout 레이어 1개를 정의함
# 2) forward()에 선형 레이어 - ReLU - 선형 레이어 - Dropout 순서의 연산을 정의함

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, config):
        """Write your code"""

    def forward(self, x):
        """
        Args:
            x: [B, seq_len, n_head*d_head]
        """
        """Write your code"""

        return x

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## 6. Decoder 레이어 구현하기

GPT는 여러 디코더 레이어를 쌓아 만드는 모델입니다. 여기선 이 디코더 레이어를 정의해보겠습니다. 데이터의 흐름은 아래와 같습니다.

1. 입력 데이터에 Layer Normalization과 Self-Attention을 진행함
    - Self-Attention은 앞선 `MultiHeadAttention` 클래스 이용
2. (1)과 입력 데이터의 residual connection을 진행함 (두 값을 더함)
3. (2)의 결과에 Layer Normalization과 FFN을 진행함
    - FFN은 앞선 `PositionwiseFeedForward` 클래스 이용
4. (2)와 (3)의 residual connection을 진행함 (두 값을 더함)

In [ ]:
############################################################################
# Req 3-7: Decoder 레이어 구현하기                                            #
############################################################################

class DecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()

        ################################################################################
        # TODO: 조건에 맞게 모델의 레이어 및 묘듈을 정의함                                   #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) MultiHeadAttention 모듈을 선언함
        # 2) layer_norm 2개를 정의함
        # 3) FeedFoward 모듈을 선언함

        self.config = config

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): shape - [B, seq_len, n_head*d_head]
        Returns:
            x (torch.Tensor): shape - [B, seq_len, n_head*d_head]
        """
        ################################################################################
        # TODO: Decoder layer의 연산을 정의함                                            #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 입력 데이터에 Layer Normalization - Self-Attention을 진행함
        # 2) (1)과 입력 데이터의 residual connection을 진행함 (두 값을 더함)
        # 3) (2)의 결과에 Layer Normalization - FFN을 진행함
        # 4) (2)와 (3)의 residual connection을 진행함 (두 값을 더함)

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return x

## 7. GPT 클래스 구현하기

최종 모델인 `GPT` 클래스는 배치 입력을 받아 모델의 차원으로 바꾸고, 포지셔널 인코딩을 더하고, 트랜스포머 레이어를 거쳐, 최종적으로 각 단어의 확률값을 구합니다. 가장 높은 확률을 가지는 단어가 다음에 올 단어라는 추론이 가능합니다.

In [ ]:
############################################################################
# Req 3-7: GPT 클래스 구현하기                                               #
############################################################################

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config

        ################################################################################
        # TODO: 조건에 맞게 모델의 레이어를 정의함                                          #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) n_dec_vocab 개수의 데이터 각각을 d_model 차원으로 표현할 임베딩을 할 레이어 정의. `nn.Embedding` 이용
        # 2) Positional encoding을 진행할 임베딩 레이어 정의. `nn.Embedding` 이용
        # 3) DecoderLayer를 n_layer번 반복하여 선언
        # 4) Layer Normalization 레이어 선언
        # 5) d_model 차원의 model feature를 vocabulary 개수 차원의 벡터로 projection할 레이어 정의.

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################


    def forward(self, x):
        """
        Args:
            x: shape - [B, seq_len]
        Returns:
            logits: shape - [B, n_dec_vocab]
        """
        ################################################################################
        # TODO: GPT 모델의 연산을 정의함                                                  #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 입력 데이터를 model feature로 임베딩함
        # 2) 포지셔널 인코딩 벡터를 구함
        # 3) 임베딩된 데이터와 포지셔널 인코딩 벡터를 더함
        # 4) (3)의 데이터를 디코더 레이어를 통과시킴
        # 5) Layer normalization을 거침
        # 6) Linear projection을 통해 입력 데이터의 다음으로 올 vocab으로 각 vocab의 점수를 구함

        """Write your code"""

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return logits

    @torch.no_grad()
    def generate(self, x, max_new_tokens):
        """
        Generate new sequence from input.
        Code from https://www.youtube.com/watch?v=kCc8FmEb1nY&t=0s

        Args:
            x (torch.Tensor): shape - [B, len_input_seq]
            max_new_tokens (int): maximum number of token generations
        Returns:
            gen_idx (torch.Tensor): token indices of generated sequence. shape - [B, len_gen_seq]

        """
        gen_idx = x
        for _ in range(max_new_tokens):
            # 1. Input of GPT. Use the latest words.
            x_cond = gen_idx[:, -self.config.seq_len:]

            # 2. Prediction of GPT
            logits = self(x_cond)

            # 3. Extract only the last time step
            logits = logits[:, -1, :]

            # 4. Softmax to get probabilities
            probs = F.softmax(logits, dim=-1)

            # 5. Sample from the distribution. shape - [B, 1]
            idx_next = torch.multinomial(probs, num_samples=1)

            # 6. Accumulate the generated results. shape - [B, T+1]
            gen_idx = torch.cat((gen_idx, idx_next), dim=1)

        return gen_idx

# 8. GPT pre-train 함수 구현하기

위에서 구현한 GPT를 학습하는 함수를 구현하겠습니다. 학습 함수 구현은 Sub PJT 1의 이미지 분류기 실습을 참고하면 좋습니다.

In [ ]:
############################################################################
# Req 3-8: GPT pre-train 함수 구현하기                                       #
############################################################################

def train(cfg, model, train_dataset, optimizer, device, log_interval=100):
    ################################################################################
    # TODO: 정의한 GPT 모델과 불러온 데이터셋을 사용하여 모델을 학습하는 함수를 구현함.       #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # 1) 모델을 train 모드로 설정
    # 2) max_iters까지 아래 과정을 반복
    #    (1) Optimizer를 초기화
    #    (2) 데이터 배치 가져오기 & 디바이스 할당하기: 앞서 구현한 `get_batch` 함수 사용
    #    (3) 모델 추론 결과 얻기: 다음 단어가 각 vocabulary에 해당할 점수를 출력함.
    #    (4) loss 계산하기: 정답과 추론 결과 간의 교차 엔트로피 오차 사용. `F.cross_entropy` 함수를 사용
    #    (5) Backpropagation으로 gradient 자동 계산하기
    #    (6) Optimzier로 모델 파라미터 업데이트
    #    (7) 현재 loss 출력 (선택사항)

    """Write your code"""

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################

## 9. 학습 및 추론하기

아래 함수는 학습한 GPT 모델로 텍스트를 생성하는 함수입니다.

In [ ]:
def gen_text(model, int_to_str, max_new_tokens=100, device='cuda'):
    # Input data
    x = torch.zeros((1, 1), dtype=torch.long, device=device)

    # Inference
    model.eval()
    gen_seq = model.generate(x, max_new_tokens)[0].tolist()
    decoded_text = decode(int_to_str, gen_seq)

    return decoded_text

앞선 모든 내용을 바탕으로 이제 모델을 학습하고 텍스트를 생성해보겠습니다.

In [ ]:
############################################################################
# Req 3-9: 학습 및 추론하기                                                   #
############################################################################

################################################################################
# TODO: GPT 모델을 불러와 pre-train을 진행함. 학습 전/후 생성되는 텍스트를 비교함       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) GPT 모델 및 optimizer (Adam) 선언
# 2) 학습 전에 생성되는 텍스트를 확인함
# 3) 모델의 학습을 진행함
# 4) 학습 후 생성되는 텍스트를 확인함

"""Write your code"""

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################